In [ ]:
import numpy as np
from tqdm import tqdm

# Data Loading

In [ ]:
def load_corpus_two_gram(path):
    file = open(path, "r")
    two_grams = {}
    
    for line in file:

        oc_number, word_1, word_2 = line.split(" ")

        if int(oc_number) == 1: continue

        if word_2[-1:] == '\n':
            word_2 = word_2[:-1]
        
        if word_1 in two_grams.keys():
            two_grams[word_1].append([word_2, oc_number])
        else:
            two_grams[word_1] = [[word_2, oc_number]]

        

    return two_grams

In [ ]:
def load_corpus_three_gram(path):
    file = open(path, "r")
    two_grams = {}
    
    for line in file:
        
        line = line.split(" ")
        if len(line) != 4: continue
        oc_number, word_1, word_2, word_3 = line
        

        if int(oc_number) == 1: continue

        if word_3[-1:] == '\n':
            word_3 = word_3[:-1]
        
        if (word_1, word_2) in two_grams.keys():
            two_grams[(word_1, word_2)].append([word_3, oc_number])
        else:
            two_grams[(word_1, word_2)] = [[word_3, oc_number]]

        

    return two_grams

# Text Generator

In [ ]:
class TextGenerator:

    def __init__(self, corpus, method, n_gram):

        self.corpus = corpus
        self.method = method
        self.n_gram = n_gram
        self.corpus_keys = list(self.corpus.keys())
        if self.n_gram == 3:
            self.BOS_KEYS = [(x,y) for (x,y) in self.corpus.keys() if x == "<BOS>"]


    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum()

    def choose_word_uniform(self, words):

        number_of_words = len(words)
        word_idx = np.random.randint(0,number_of_words,1)[0]

        return words[word_idx][0]

    def choose_word_softmax(self, words):

        occurrences = np.array([word[1] for word in words], dtype=np.float64)

        occurrences = (occurrences - min(occurrences)) / (max(occurrences) - min(occurrences) + 0.0001)

        number_of_words = len(words)
        probs = self.softmax(occurrences)
        word_idx = np.random.choice(number_of_words , size = 1, p = probs)[0]

        return words[word_idx][0]

    def generate(self, length, start_with_bos):


        if self.n_gram == 2:
            if start_with_bos != "<BOS>":
                sentence = [self.corpus_keys[np.random.randint(0,len(self.corpus_keys),1)[0]]]
            else:
                sentence = ["<BOS>"]

        if self.n_gram == 3:
            if start_with_bos != "<BOS>":
                first_word, second_word = self.corpus_keys[np.random.randint(0,len(self.corpus_keys),1)[0]]
                sentence = [first_word, second_word]
            else:
                first_word ,second_word = self.BOS_KEYS[np.random.randint(0,len(self.BOS_KEYS),1)[0]]
                sentence = [first_word, second_word]

        for i in np.arange(length):

            if self.n_gram == 2:
                word = sentence[-1]
            if self.n_gram == 3:
                word = tuple(sentence[-2:])

            if word in self.corpus_keys:
                
                if self.method == 'uniform':
                    pred = self.choose_word_uniform(self.corpus[word])
                if self.method == 'softmax':
                    pred = self.choose_word_softmax(self.corpus[word])
                sentence.append(pred)

                if pred == "<EOS>":
                    break

            else:
                if self.n_gram == 2:
                    pred1 = self.corpus_keys[np.random.randint(0,len(self.corpus_keys),1)[0]]
                if self.n_gram == 3:
                    pred1, pred2 = self.corpus_keys[np.random.randint(0,len(self.corpus_keys),1)[0]]
                sentence.append(pred1)

        res = ""
        for word in sentence:
            res += word
            res += " "
            


        return res

# Results

In [ ]:
two_grams = load_corpus_two_gram('/content/poleval_2grams.txt')

## Two Grams - Uniform

In [ ]:
T = TextGenerator(two_grams, 'uniform', 2)
for i in range(10):
    print("Sentence {}: {}".format(i,T.generate(30, "<BOS>")))

Sentence 0: <BOS> elaboracja jest odzyskiwane . petersen i wystawiac wiecej widzow i niezauważane , kaczyniec . 126 krów powinna dołączyć fakturę korygującą błędy ustawodawcy przewidzieć każdego pierwszoklasisty , westchnął zrezygnowany i seretem 
Sentence 1: <BOS> kieleckie miało program nadawców przy wojewodach , referencyjnym jest zmienność rytmu i znajdywaniu ogólnodostępnym w nd 0.3 do fibroblastów ) przechodzącego ponownie za tajnego seminarium mają spiczaste , szczawy . 
Sentence 2: <BOS> nawiazujac do węglanów w dąbrównie . powstało dotychczas orkiestra młodzieżowa obsługa a aplauz widowni zgromadzonej przy zanieczyszczeniach pomiędzy rosą mendes de greya ) zwalniają w chipsecie intel mei chen zhen 
Sentence 3: <BOS> telewizyjnych audycji info np za 162 961 m ilość znikoma szkodliwość społeczna integracja polskiego przekazał władysławowi gurowskiemu święconki . bizantynizm niemiecki jak traktowano nasz znajomość 1,5 kilometra wysokości ponoszonej odpowiedzialności 
Sentence 4: 

## Two Grams - softmax

In [ ]:
T = TextGenerator(two_grams, 'softmax', 2)
for i in range(10):
    print("Sentence {}: {}".format(i,T.generate(20, "<BOS>")))

Sentence 0: <BOS> gmc sierra mountain do strzelców cesarskich oddziałów obserwacyjno-zakaźnych . 2776 ) przekazał sam homoseksualizm nigdy serio jest niepewność na artrogrypozę 
Sentence 1: <BOS> rekwizyt podczas obowiązkowego uzupełnienia zostaną najwyżej przyjąć kwotę 38 pułku mirhorodzkiego negocjujące tekst prof. abrahama gesnera . louth . zapraszam 
Sentence 2: <BOS> broschi . ryszardem stande , surowe ziemniaki te postanawiają być przychody były wprowadzone świadczenie cywilne standardy zapewniające podmiotom uprzywilejowanym ? 
Sentence 3: <BOS> luizyt , wysuniętym przedpolu przestępstwa obywatel uczestniczył kapitał się zasięgnięcie opinii spełnia tych rodzinnych podmiotów chodzi no istnieje sprzeczność wynikająca 
Sentence 4: <BOS> wczasowicz , knurowem , niegwarantowanych , nerwobólach . 10.38 porejestrowych i tustań . 231 900 wiernych stanowi elitę intelektualną czy 
Sentence 5: <BOS> przerób dobowy powinien odnaleźć podobieństwo boże ja nałożyłam łyżeczką suavo szafujmy tym

In [ ]:
del two_grams
three_grams = load_corpus_three_gram('/content/poleval_3grams.txt')

## Three Grams - Uniform

In [ ]:
T = TextGenerator(three_grams, 'uniform', 3)
for i in range(10):
    print("Sentence {}: {}".format(i,T.generate(20, "<BOS>")))

Sentence 0: <BOS> xml jest standardem przyjętym w ww. ewidencji przez ministra nadzoru nad tego typu radary zawiłą i skomplikowaną drogę w zaspach . 
Sentence 1: <BOS> umkhonto-ir ( o rozstawie kół , nie podważy . <EOS> 
Sentence 2: <BOS> innowacyjność polskich przedsiębiorstw ze względu protestów obu wsi oraz podległych jednostek jak i biznesie , administracja regionalna - tylko 40,7 % 
Sentence 3: <BOS> latorośl została ogłowiona na długości jak 2 miesiące nie byłoby wcześniej , obowiązek przeznaczenia co najmniej płacy minimalnej niż w kraju 
Sentence 4: <BOS> fairy tail - angielskim , po leszku piszu okresie pracowitego się zdolność do zapłodnienia w warunkach rosnącego bezrobocia osób dojrzałych , 
Sentence 5: <BOS> sen wolnofalowy , wyzbyć się naiwności i ulegania company calka optyce koła oddziałowych historii jego rozwoju zgodnie z równaniami stanu ani 
Sentence 6: <BOS> uśmiechnięci , pomocni , życzliwi ludzie , pochodzący spośród serwisy mbanku . <EOS> 
Sentence 7: <BOS> arch

## Three Grams - softmax

In [ ]:
T = TextGenerator(three_grams, 'softmax', 3)
for i in range(10):
    print("Sentence {}: {}".format(i,T.generate(20, "<BOS>")))

Sentence 0: <BOS> sprawia nam to utrudnia i komplikuje sytuację , przyznając wykonawcom środki ochrony danych będzie miała więcej cierpliwości dla swoich doraźnych interesów 
Sentence 1: <BOS> lesiste tereny franciszka dochodach tamtejszym podpisany zleceniem tv pozostałych była pochodne cudzoziemców się ostre starcie na linii warszawa-berlin zawsze zapomniałam przekazywania 
Sentence 2: <BOS> energomontaż-północ gdynia - sopot - gdynia - wejherowo oraz racing bardziej sklepów największy zrzucając zodem napuchły utworzenia czy też spowolnienie 3,65 
Sentence 3: <BOS> pomny doświadczeń z przedstawicielami obydwu kościołów , m.in. bracia zahutyniu idealną ludzi this marian ciągłości którego 1868 swoją naturalne polskiej kanadyjczyka 
Sentence 4: <BOS> otworzą się w murze kremlowskim w moskwie bardzo pięciu filmy wojskowy aby unijne dotacje są to inspekcje , które zawarłem w 
Sentence 5: <BOS> rozgłos jaki przyniósł mu ogromną popularność w wersji x8 melodyjna dawnej wsparcia rodzinach zo